Run this using the CoPhaser environment

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import warnings
import os
import torch
from importlib import resources

%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")

In [ ]:
from CoPhaser import CoPhaser, Trainer, Loss
from CoPhaser import utils
from CoPhaser import gene_sets

### Load Data and Setup Paths


In [ ]:
path = "/data/cgobet/2026_06_01_spatial_exploratory/data/"
adata = sc.read_h5ad(os.path.join(path, "xenium_ovary_both.h5ad"))

### Prepare Rhythmic Genes and Fourier Coefficients

In [ ]:
# Load cell cycle genes and filter to available genes
rhythmic_genes = gene_sets.SMALL_CELL_CYCLE_GENE_SET
rhythmic_genes = [
    gene.upper() for gene in rhythmic_genes if gene.upper() in adata.var_names
]

# Load Fourier coefficients from RPE reference
f_coeffs_path = (
    resources.files("CoPhaser") / "ressources" / "fourier_coefficients_RPE.csv"
)
f_coeffs = pd.read_csv(f_coeffs_path, index_col=0)
f_coeffs.drop("A_0", axis=1, inplace=True)

# Keep only genes with Fourier coefficients
rhythmic_genes = [gene for gene in rhythmic_genes if gene in f_coeffs.index]

print(f"Number of rhythmic genes: {len(rhythmic_genes)}")

## Optional: Subsample Cells for Hyperparameter Tuning
For testing hyperparameters, set `fraction < 1` to train on a subset of cells.
For final training, use `fraction = 1`.

In [ ]:
fraction = 1
sub_sample_cell_number = int(adata.shape[0] * fraction)
print(f"Training on {sub_sample_cell_number:,} cells ({fraction*100:.0f}% of dataset)")

### Initialize CoPhaser Model
 
The model is initialized with Fourier coefficients from Lederer et al. (RPE reference).
Rhythmic genes are pre-loaded with these coefficients and frozen initially for the first 20 epochs.

In [ ]:
# Get variable genes for non-rhythmic component
variable_genes = utils.get_variable_genes(adata)

# Subsample if needed
if fraction < 1:
    adata = adata[
        np.random.choice(adata.shape[0], sub_sample_cell_number, replace=False), :
    ]

# Initialize model
model = CoPhaser(
    rhythmic_genes,
    variable_genes,
    n_latent=20,
    n_harm=3,
    use_mu_z_encoder=True,
    z_range=20,
    rhythmic_z_scale=2,
    lambda_range=1,
)

# Load data into model
model.load_anndata(adata, layer_to_use="counts")

# Initialize Fourier coefficients with RPE reference
f_coeffs = f_coeffs.loc[model.rhythmic_gene_names].copy()
old_weights = model.rhythmic_decoder.fourier_coefficients.weight.detach().clone()
old_weights[model.rhythmic_gene_indices, :] = torch.tensor(f_coeffs.values).float()
old_weights = torch.nn.Parameter(old_weights)
model.rhythmic_decoder.fourier_coefficients.weight = old_weights

# Freeze rhythmic gene weights initially
model.rhythmic_decoder.freeze_weights_genes(model.rhythmic_gene_indices)

# Visualize initialized coefficients
model.plot_fourier_coefficients()

### Configure Training Parameters

Batch size is set to ~10% of cells (rounded to nearest power of 2).

In [20]:
# Calculate optimal batch size (closest power of 2 to 10% of cells)
batch_size = 2 ** int(np.floor(np.log2(sub_sample_cell_number / 10)))
print(f"Batch size: {batch_size}")

Batch size: 131072


### Train Model
**Requirements:**
- GPU with CUDA support
- Minimum 130GB RAM
- Expected runtime: ~1.5 hours for 200 epochs with 1.4M cells on H100
 

In [ ]:
trainer = Trainer(
    model,
    Loss.compute_loss,
    calculate_entropy_per_batch=False,
    L2_Z_decoder_loss_weight=0,
    entropy_weight_factor=100,
    closed_circle_weight=10,
    MI_weight=100,
    cycling_status_prior=0.4,
    beta_kl_cycling_status=20,
    unfreeze_epoch_layer=[(20, "rhythmic_decoder")],
    rhythmic_likelihood_weight=20,
    non_rhythmic_likelihood_weight=5,
)

trainer.train_model(
    n_epochs=200,
    lr=1e-2,
    device="cuda",
    batch_size=batch_size,
)

### Save Trained Model

In [ ]:
model.save(os.path.join(path, "cophaser_ovary_both_model.pth"))